# Automação Web e Busca de Informações com Python

#### Desafio: 

Automação de atualização de preços a partir dos valores de cotação.

In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#abrir o navegador
navegador = webdriver.Chrome()
navegador.get("https://www.google.com.br/")

#localizar o campo de busca na página (vale para qualquer elemento do site - inspecionar e copiar o xpath)
#Cotação dolar

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#Obter/extrair as informações da página
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação dolar hoje:", cotacao_dolar)

#Cotação euro
navegador.get("https://www.google.com.br/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print("Cotação euro hoje:", cotacao_euro)

#Cotação ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print("cotação ouro hoje:", cotacao_ouro)

#Fechando o navegador
navegador.quit()


Cotação dolar hoje: 4.9345
Cotação euro hoje: 5.889863610000001
cotação ouro hoje: 282.62


### tualizar a base de preços com as novas cotações

- Importando a base de dados

In [27]:
#Importar a lista de produtos

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizar a cotação
- Atualizando o preço Base Reais (preço base original * cotação)
- Atualizar o Preço Final (preço base reais * margem)

In [28]:
#atualizar a cotação
#localização da cotação dolar - linhas onde a coluna Moeda é Dólar e coluna Cotação

tabela.loc[tabela["Moeda"] == "Dólar" ,"Cotação"] = float(cotacao_dolar)

#o mesmo para as outras cotações
tabela.loc[tabela["Moeda"] == "Euro" ,"Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro" ,"Cotação"] = float(cotacao_ouro)

#atualizar preço base reais

tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

#atualziar preço final

tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.934500,4934.450655,1.40,6908.230917
1,Carro Renault,4500.00,Euro,5.889864,26504.386245,2.00,53008.772490
2,Notebook Dell,899.99,Dólar,4.934500,4441.000655,1.70,7549.701113
3,IPhone,799.00,Dólar,4.934500,3942.665500,1.70,6702.531350
4,Carro Fiat,3000.00,Euro,5.889864,17669.590830,1.90,33572.222577
5,Celular Xiaomi,480.48,Dólar,4.934500,2370.928560,2.00,4741.857120
6,Joia 20g,20.00,Ouro,282.620000,5652.400000,1.15,6500.260000


### Exportar a nova base de preços atualizada

In [29]:
tabela.to_excel("Produtos Novos.xlsx", index=False)